In [7]:
import mysql.connector
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from pandas import Timestamp
from IPython.display import display
from Functions.connection.connection import *
from Functions.AlertP1.data_cleaning import *
from Functions.AlertP1.features import *
from Functions.analysis.step_analysis import *
from Functions.AlertP1.dummy_features import *
from Functions.Models.decision_tree import *
from Functions.Models.Logistic_regression import *
from Functions.Models.evaluation import *
import spacy
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from numpy import argsort
from Functions.AlertP1.data_cleaning import *
from Functions.AlertP1.features import *
from Functions.analysis.step_analysis import *
from Functions.AlertP1.dummy_features import *
from Functions.Models.evaluation import *
from Functions.NLP.alertp1_nlp import *
from Functions.NLP.data_with_nlp import *
from Functions.pipeline import *
from gensim.models import Word2Vec
import re
import nltk
import gensim
import math

In [8]:
creds = ["uktuserdaroglu","Vacy4201(Pri","172.20.20.4","hgo",3306]
#Connection to the database
host = creds[2]
user = creds[0]
password = creds[1]
database = creds[3]
port = creds[4]
mydb = mysql.connector.connect(host=host, user=user, database=database, port=port, password=password, auth_plugin='mysql_native_password')
mycursor = mydb.cursor()

#Safecheck to guarantee that the connection worked
mycursor.execute('SHOW TABLES;')
print(f"Tables: {mycursor.fetchall()}")
print(mydb.connection_id) #it'll give connection_id,if got connected

Tables: [('ConsultaUrgencia_doentespedidosconsultaNeurologia2012',), ('consultaneurologia2012',), ('consultaneurologia201216anon_true',), ('hgo_data_032023',)]
233


In [9]:
alertP1 = pd.read_sql("""SELECT * FROM consultaneurologia201216anon_true""",mydb)

/Users/utkuserdaroglu/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [5]:
def w2v_v1(alertP1):
    alertP1= pre_process(alertP1)
    sentences = alertP1['clean_text'].tolist()
    model = Word2Vec(sentences, window=3, min_count=5, workers=4,sg=1,alpha=0.01)  # Adjust parameters as needed
    #Featurization
    def get_sentence_vector(sentence):
        vectors = []
        for word in sentence:
            if word in model.wv:
                vectors.append(model.wv[word])
        if vectors:
            return np.mean(vectors, axis=0)
        else:
            return np.zeros(model.vector_size)
        
    alertP1['word2vec_feature'] = alertP1['clean_text'].apply(lambda x: get_sentence_vector(x)) #assigning w2v to the correct columns.
    return(alertP1)

In [6]:
w2v(alertP1)

/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/data_cleaning.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['PROVENIENCIA'][alertP1['PROVENIENCIA']=='']='unknown'
/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/data_cleaning.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['CTH_PRIOR'][alertP1['CTH_PRIOR']=='']='unknown'
/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/data_cleaning.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydat

,ID_DOENTE,PROCESSO,COD_REFERENCIA,COD_PZ,COD_UNID_SAUDE_PROV,UNID_PROV,TIPO_UNID,COD_CTH_PRIOR,CTH_PRIOR,COD_MOTIVO_RECUSA,...,unknown,Other specialities,2,3+,HOSP,UCSP,USF A,USF B,outro,word2vec_feature
1499,01-12-1GBF,MIVNNVE,LPHRRVQP,,0,,OUTRA,,unknown,0,...,0,0,0,0,0,0,0,0,0,"[-0.07447971, 0.14504914, 0.029223422, 0.12511..."
959,0F-12-1GAG,LLCCECB,LRHHVTQR,GAGBESD,3151571,USF FFMAIS-CS SEIXAL,CS/USF,,unknown,7,...,0,0,0,0,0,0,0,1,0,"[-0.07290505, 0.13053244, 0.041477233, 0.12394..."
868,0F-12-1GAS,LLCJILJ,LQHOOLQV,,0,,OUTRA,,unknown,0,...,0,0,0,0,0,0,0,0,0,"[-0.08673388, 0.14699431, 0.031748634, 0.10514..."
1519,0S-12-1GBF,MIVEMLC,LRHPVSVU,ZHEEZFSC,3151571,USF FFMAIS-CS SEIXAL,CS/USF,3,Normal,0,...,0,0,0,0,0,0,0,1,0,"[-0.07619329, 0.13393104, 0.04039456, 0.125927..."
1054,"A,CE+SF",LLDMBND,LSHPTSTP,,0,,OUTRA,,unknown,7,...,0,0,0,0,0,0,0,0,0,"[-0.09036544, 0.13468635, 0.02850105, 0.115123..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906,ZZHAZB,LLCIVVM,LOHHPTRS,SBDCHFZ,3150502,CHARNECA DA CAPARICA,CS/USF,,unknown,7,...,0,0,0,0,0,0,1,0,0,"[-0.08012816, 0.13918495, 0.03255802, 0.124374..."
44,ZZHFGC,LVCBJE,LPHOQTHL,AGEDDZE,3150671,USF COVA DA PIEDADE-CS C PIEDA,CS/USF,,unknown,0,...,0,1,0,0,0,0,0,1,0,"[-0.080602124, 0.14182073, 0.028439002, 0.1223..."
49,ZZHHSA,LVDLVJ,LQHHOTVL,,0,,OUTRA,,unknown,53,...,0,0,0,0,0,0,0,0,0,"[-0.07601211, 0.14593154, 0.03640728, 0.120280..."
255,ZZSSDF,LLBCCC,LQHLSPPQ,BDEEFAS,3150571,USF SOBREDA-CS COSTA CAPARICA,CS/USF,,unknown,0,...,0,0,0,0,0,0,0,1,0,"[-0.07849052, 0.14083242, 0.04156389, 0.119242..."


In [10]:
def w2v(alertP1):
    alertP1= pre_process(alertP1)
    
    # Split data into train and test
    AlertP1_sorted = alertP1[alertP1['clean_text']!=''].sort_values(by='DATA_RECEPCAO')

    # calculate the index for the split
    split_index = math.ceil(0.8 * len(AlertP1_sorted))

    # split the data frame into test and train sets
    train_set = AlertP1_sorted.iloc[:split_index]
    test_set = AlertP1_sorted.iloc[split_index:]


    #Converting text into list of sentences
    sentences = train_set['clean_text'].tolist()

    #W2V model building
    model = Word2Vec(sentences, window=3, min_count=5, workers=4,sg=1,alpha=0.01)  # Adjust parameters as needed
    
    #Featurization
    def get_sentence_vector(sentence):
        vectors = []
        for word in sentence:
            if word in model.wv:
                vectors.append(model.wv[word])#If the word in the text exists in the W2V vocabulary, it assigns the vector 
        if vectors:
            return np.mean(vectors, axis=0)#Takes the mean of the vectors for that referral
        else:
            return np.zeros(model.vector_size) #if it's unsuccesful to find the word in the dictionary, then 0 
        
    alertP1['word2vec_feature'] = alertP1['clean_text'].apply(lambda x: get_sentence_vector(x)) #assigning w2v to the correct columns.
    return(alertP1)

In [11]:
w2v_v2(alertP1)

/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/data_cleaning.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['PROVENIENCIA'][alertP1['PROVENIENCIA']=='']='unknown'
/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/data_cleaning.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['CTH_PRIOR'][alertP1['CTH_PRIOR']=='']='unknown'
/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/data_cleaning.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydat

,ID_DOENTE,PROCESSO,COD_REFERENCIA,COD_PZ,COD_UNID_SAUDE_PROV,UNID_PROV,TIPO_UNID,COD_CTH_PRIOR,CTH_PRIOR,COD_MOTIVO_RECUSA,...,unknown,Other specialities,2,3+,HOSP,UCSP,USF A,USF B,outro,word2vec_feature
1499,01-12-1GBF,MIVNNVE,LPHRRVQP,,0,,OUTRA,,unknown,0,...,0,0,0,0,0,0,0,0,0,"[-0.09569886, 0.12704776, 0.03180352, 0.116788..."
959,0F-12-1GAG,LLCCECB,LRHHVTQR,GAGBESD,3151571,USF FFMAIS-CS SEIXAL,CS/USF,,unknown,7,...,0,0,0,0,0,0,0,1,0,"[-0.09467662, 0.1124859, 0.045338, 0.1144669, ..."
868,0F-12-1GAS,LLCJILJ,LQHOOLQV,,0,,OUTRA,,unknown,0,...,0,0,0,0,0,0,0,0,0,"[-0.109628014, 0.120068386, 0.037116963, 0.096..."
1519,0S-12-1GBF,MIVEMLC,LRHPVSVU,ZHEEZFSC,3151571,USF FFMAIS-CS SEIXAL,CS/USF,3,Normal,0,...,0,0,0,0,0,0,0,1,0,"[-0.09888068, 0.11380833, 0.043685466, 0.11633..."
1054,"A,CE+SF",LLDMBND,LSHPTSTP,,0,,OUTRA,,unknown,7,...,0,0,0,0,0,0,0,0,0,"[-0.10772862, 0.111654766, 0.03422259, 0.09948..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906,ZZHAZB,LLCIVVM,LOHHPTRS,SBDCHFZ,3150502,CHARNECA DA CAPARICA,CS/USF,,unknown,7,...,0,0,0,0,0,0,1,0,0,"[-0.10168219, 0.113870025, 0.03781807, 0.11425..."
44,ZZHFGC,LVCBJE,LPHOQTHL,AGEDDZE,3150671,USF COVA DA PIEDADE-CS C PIEDA,CS/USF,,unknown,0,...,0,1,0,0,0,0,0,1,0,"[-0.100633286, 0.1185576, 0.034708716, 0.11001..."
49,ZZHHSA,LVDLVJ,LQHHOTVL,,0,,OUTRA,,unknown,53,...,0,0,0,0,0,0,0,0,0,"[-0.09632244, 0.1221355, 0.04308398, 0.1131943..."
255,ZZSSDF,LLBCCC,LQHLSPPQ,BDEEFAS,3150571,USF SOBREDA-CS COSTA CAPARICA,CS/USF,,unknown,0,...,0,0,0,0,0,0,0,1,0,"[-0.10149653, 0.118138134, 0.044570312, 0.1117..."
